In [ ]:
# Всё о средствах авторизации в сервисах Яндекса

При развертке на сервисах яндекса **iam** ключ доступен как переменная среды

## 1. Общие понятия: JWT, IAM, OAuth

### Что такое JWT?

**JWT (JSON Web Token)** — это стандарт для безопасной передачи информации между сторонами в виде JSON-объекта. Он может быть подписан (с помощью секрета или ключей) и/или зашифрован.

Структура JWT:
>header.payload.signature

В Yandex Cloud:
* JWT не используется напрямую клиентами, но внутри системы он применяется при обмене токенами.
* Для внешних API используются OAuth-токены, IAM-токены, сервисные аккаунты, ключ доступа (access key + secret key).

## 2. Генерация JWT-токена (для получения IAM-токена)

**JWT** используется временно для получения **IAM-токена** от **Yandex Cloud** через **OAuth-совместимый эндпоинт**. <br>

**Когда нужно генерировать JWT?**<br>
Когда вы используете сервисный аккаунт и хотите получить временный IAM-токен без постоянного хранения OAuth-токена.

#### Шаги:

##### 1. Создайте сервисный аккаунт

* В консоли Yandex Cloud создайте сервисный аккаунт.
* Назначьте ему роли (например, editor, storage.admin, ai.languageModels.user и т.д.).

##### 2. Сгенерируйте пару ключей

* В интерфейсе сервисного аккаунта создайте ключ для входа → формат: JSON.
* Вы получите файл с:
    * service_account_id
    * key_algorithm (обычно RSA-2048)
    * private_key (в PEM-формате)
    * public_key_fingerprint
    * id (идентификатор ключа)

##### 3. Сформируйте JWT

**Header:**

In [ ]:
{
  "alg": "PS256",
  "kid": "KEY_ID"  // id из JSON-ключа
}

**Payload:**

In [ ]:
{
  "aud": "https://iam.api.cloud.yandex.net/iam/v1/tokens",
  "iss": "SERVICE_ACCOUNT_ID",
  "sub": "SERVICE_ACCOUNT_ID",
  "iat": UNIX_TIMESTAMP,
  "exp": UNIX_TIMESTAMP + 3600  // не более часа
}

##### 4. Подпишите JWT с помощью приватного ключа

Используйте алгоритм **PS256** (RSA с SHA-256 и PSS padding).

Пример на Python:

In [ ]:
import jwt
import time

def generate_jwt(service_account_id, key_id, private_key):
    now = int(time.time())
    payload = {
        'aud': 'https://iam.api.cloud.yandex.net/iam/v1/tokens',
        'iss': service_account_id,
        'sub': service_account_id,
        'iat': now,
        'exp': now + 3600
    }
    encoded_jwt = jwt.encode(
        payload,
        private_key,
        algorithm='PS256',
        headers={'kid': key_id}
    )
    return encoded_jwt

## 3. Получение IAM-токена по JWT

Отправьте JWT на эндпоинт:

In [ ]:
POST https://iam.api.cloud.yandex.net/iam/v1/tokens
Content-Type: application/json

{
  "jwt": "ваш_подписанный_JWT"
}

Ответ:

In [ ]:
{
  "iamToken": "t1.9e...",
  "expiresAt": "2024-01-01T00:00:00Z"
}

Этот **iamToken** — временный токен (действует до 12 часов), который можно использовать для вызова API.

## 4. Варианты авторизации в Yandex Cloud

|метод| назначение| срок жизни|
|-|-|-|
|IAM-токен|Основной способ для API|До 12 часов|
|OAuth-токен|Пользовательская авторизация|1 год (может быть отозван)|
|Ключ доступа (Access Key + Secret)|Для S3-совместимых сервисов (YOS)|Бессрочный (если не удалить)|
|Service Account + JWT → IAM Token|Автоматизация, серверы|Через JWT каждый час|

## 5. Доступ к Yandex GPT

**Требуется:** <br>
IAM-токен (временный) <br>
Или статический OAuth-токен (не рекомендуется для продакшена) <br>

**Эндпоинт:**
> POST https://llm.api.cloud.yandex.net/foundationModels/v1/completion

**Заголовки:**
>Authorization: Bearer \< IAM-TOKEN \> <br>
>Content-Type: application/json

**Пример запроса:**

In [ ]:
{
  "modelUri": "gpt://<folder-id>/yandexgpt/latest",
  "completionOptions": {
    "stream": false,
    "temperature": 0.6,
    "maxTokens": "1000"
  },
  "messages": [
    {
      "role": "system",
      "text": "Ты помощник."
    },
    {
      "role": "user",
      "text": "Привет!"
    }
  ]
}

## 6. Доступ к Yandex Object Storage (YOS)
YOS — S3-совместимое хранилище. Авторизация возможна двумя способами:

Ключ доступа (S3-style)

**Шаги:**
1. Создайте ключ доступа для сервисного аккаунта (в консоли).
1. Получите:
    * Access Key ID (публичная часть)
    * Secret Access Key (приватная)

#### Конфигурация клиента (boto3, AWS CLI):

In [ ]:
import boto3

s3 = boto3.client(
    's3',
    endpoint_url='https://storage.yandexcloud.net',
    aws_access_key_id='YCA...ID',
    aws_secret_access_key='YCK...KEY'
)

## Автоматизация: Как обновлять токены?
**Для GPT:**
* Раз в час генерируйте новый JWT.
* Получайте свежий IAM-токен.
* Используйте его для запросов к GPT.

**Для YOS:**
* Если используете ключ доступа — он постоянный.
* Если нужна повышенная безопасность — можно вращать ключи программно через IAM API.